# Singleton Pattern

## Intent
Ensure a class has only one instance and provide a global point of access to it.

## Problem
Sometimes you need exactly one instance of a class:
- Configuration manager
- Logger
- Database connection pool
- Thread pool
- Cache

Multiple instances would cause problems:
- Conflicting configuration
- Resource waste
- Synchronization issues

## When to Use
✅ **Use when:**
- Exactly one instance must exist
- Instance must be accessible from anywhere
- Lazy initialization is beneficial

❌ **Avoid when:**
- You just want a "global variable"
- Makes unit testing difficult
- Hides dependencies
- Can cause tight coupling

## Example 1: Configuration Manager (Without Singleton)

**Problem**: Multiple config instances with different values

In [ ]:
# WITHOUT Singleton - Problems arise
class ConfigManager:
    def __init__(self):
        self.settings = {}
    
    def set(self, key, value):
        self.settings[key] = value
    
    def get(self, key):
        return self.settings.get(key)

# Problem: Multiple instances with different values!
config1 = ConfigManager()
config1.set('api_key', 'secret123')

config2 = ConfigManager()  # Different instance!
print(f"Config2 api_key: {config2.get('api_key')}")  # None! Lost the setting

print(f"Are they the same? {config1 is config2}")  # False - different objects

## Implementation 1: Using Metaclass (Most Pythonic)

Metaclasses control class creation in Python.

In [ ]:
import threading

class SingletonMeta(type):
    """
    Thread-safe Singleton metaclass.
    """
    _instances = {}
    _lock = threading.Lock()
    
    def __call__(cls, *args, **kwargs):
        # Double-checked locking for thread safety
        if cls not in cls._instances:
            with cls._lock:
                if cls not in cls._instances:
                    instance = super().__call__(*args, **kwargs)
                    cls._instances[cls] = instance
        return cls._instances[cls]


class ConfigManager(metaclass=SingletonMeta):
    def __init__(self):
        # Only runs once!
        self.settings = {}
        print("ConfigManager initialized!")
    
    def set(self, key, value):
        self.settings[key] = value
    
    def get(self, key):
        return self.settings.get(key)


# Test it
print("Creating config1...")
config1 = ConfigManager()
config1.set('api_key', 'secret123')
config1.set('debug', True)

print("\nCreating config2...")
config2 = ConfigManager()  # Uses existing instance!
print(f"Config2 api_key: {config2.get('api_key')}")  # Has the value!
print(f"Config2 debug: {config2.get('debug')}")

print(f"\nAre they the same? {config1 is config2}")  # True!
print(f"ID config1: {id(config1)}")
print(f"ID config2: {id(config2)}")

## Implementation 2: Using Decorator

In [ ]:
def singleton(cls):
    """Decorator to make a class a Singleton."""
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance


@singleton
class Logger:
    def __init__(self):
        self.logs = []
        print("Logger initialized!")
    
    def log(self, message):
        self.logs.append(message)
        print(f"LOG: {message}")
    
    def get_logs(self):
        return self.logs


# Test it
logger1 = Logger()
logger1.log("First message")

logger2 = Logger()
logger2.log("Second message")

print(f"\nLogger1 logs: {logger1.get_logs()}")
print(f"Logger2 logs: {logger2.get_logs()}")
print(f"Same instance? {logger1 is logger2}")

## Implementation 3: Using __new__ Method

In [ ]:
class DatabaseConnection:
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            print("Creating new database connection...")
            cls._instance = super().__new__(cls)
            # Initialize connection here
            cls._instance.connection_string = "postgresql://localhost:5432/mydb"
            cls._instance.connected = True
        return cls._instance
    
    def execute(self, query):
        return f"Executing: {query} on {self.connection_string}"


# Test it
db1 = DatabaseConnection()
print(db1.execute("SELECT * FROM users"))

db2 = DatabaseConnection()  # Reuses existing
print(f"\nSame connection? {db1 is db2}")
print(f"Both connected? {db1.connected and db2.connected}")

## Implementation 4: Module-Level Singleton (Pythonic Way)

In Python, modules are singletons by nature!

In [ ]:
# This would typically be in a separate file like 'config.py'
# Just create an instance at module level

class _ConfigManager:
    def __init__(self):
        self.settings = {}
    
    def set(self, key, value):
        self.settings[key] = value
    
    def get(self, key):
        return self.settings.get(key)

# Create single instance at module level
config = _ConfigManager()

# Usage: import config
# config.set('key', 'value')

print("Module-level singleton created!")
config.set('module_level', True)
print(f"Setting: {config.get('module_level')}")

## Real-World Example: Application Settings Manager

In [ ]:
import json
from pathlib import Path

class Settings(metaclass=SingletonMeta):
    """Application settings manager."""
    
    def __init__(self):
        self._settings = {
            'app_name': 'MyApp',
            'version': '1.0.0',
            'debug': False,
            'database': {
                'host': 'localhost',
                'port': 5432,
                'name': 'myapp'
            }
        }
        print("Settings initialized with defaults")
    
    def get(self, key, default=None):
        """Get setting value."""
        keys = key.split('.')
        value = self._settings
        for k in keys:
            if isinstance(value, dict):
                value = value.get(k)
                if value is None:
                    return default
            else:
                return default
        return value
    
    def set(self, key, value):
        """Set setting value."""
        keys = key.split('.')
        current = self._settings
        for k in keys[:-1]:
            if k not in current:
                current[k] = {}
            current = current[k]
        current[keys[-1]] = value
    
    def load_from_dict(self, settings_dict):
        """Load settings from dictionary."""
        self._settings.update(settings_dict)
    
    def to_dict(self):
        """Export settings as dictionary."""
        return self._settings.copy()


# Usage across the application
print("\n--- Application Module 1 ---")
settings = Settings()
settings.set('debug', True)
settings.set('database.host', '192.168.1.100')

print(f"App name: {settings.get('app_name')}")
print(f"Debug mode: {settings.get('debug')}")
print(f"DB host: {settings.get('database.host')}")

print("\n--- Application Module 2 ---")
# Another part of the application
settings2 = Settings()  # Gets same instance!
print(f"Debug mode: {settings2.get('debug')}")  # Still True
print(f"DB host: {settings2.get('database.host')}")  # Still 192.168.1.100

print(f"\nSame settings instance? {settings is settings2}")

## Real-World Example: Connection Pool Manager

In [ ]:
import time
from queue import Queue

class ConnectionPool(metaclass=SingletonMeta):
    """Database connection pool manager."""
    
    def __init__(self, max_connections=5):
        self.max_connections = max_connections
        self.available = Queue(maxsize=max_connections)
        self.in_use = set()
        
        # Initialize connections
        for i in range(max_connections):
            conn = f"Connection-{i+1}"
            self.available.put(conn)
        
        print(f"ConnectionPool initialized with {max_connections} connections")
    
    def acquire(self):
        """Get a connection from the pool."""
        if not self.available.empty():
            conn = self.available.get()
            self.in_use.add(conn)
            print(f"  Acquired {conn}")
            return conn
        else:
            raise Exception("No connections available!")
    
    def release(self, conn):
        """Return a connection to the pool."""
        if conn in self.in_use:
            self.in_use.remove(conn)
            self.available.put(conn)
            print(f"  Released {conn}")
    
    def status(self):
        """Show pool status."""
        print(f"\nPool Status:")
        print(f"  Available: {self.available.qsize()}")
        print(f"  In use: {len(self.in_use)}")
        print(f"  Total: {self.max_connections}")


# Simulate database operations
print("--- Task 1 ---")
pool1 = ConnectionPool(max_connections=3)
conn1 = pool1.acquire()
conn2 = pool1.acquire()
pool1.status()

print("\n--- Task 2 (different part of app) ---")
pool2 = ConnectionPool()  # Same pool!
print(f"Is same pool? {pool1 is pool2}")
pool2.status()  # Shows same status!

print("\n--- Releasing connections ---")
pool2.release(conn1)
pool1.release(conn2)
pool1.status()

## Thread Safety Test

In [ ]:
import threading
import time

class Counter(metaclass=SingletonMeta):
    def __init__(self):
        self.count = 0
        print(f"Counter initialized by thread {threading.current_thread().name}")
    
    def increment(self):
        self.count += 1


def worker(worker_id):
    counter = Counter()
    print(f"Worker {worker_id}: Counter instance {id(counter)}")
    counter.increment()


# Create multiple threads
threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

# Wait for all threads
for t in threads:
    t.join()

# Verify single instance
final_counter = Counter()
print(f"\nFinal count: {final_counter.count}")  # Should be 5
print("All threads used the same instance!")

## Advantages & Disadvantages

### ✅ Advantages
1. **Controlled access**: Single instance is controlled
2. **Reduced namespace**: No global variables
3. **Lazy initialization**: Created only when needed
4. **Permits refinement**: Can subclass singleton
5. **Consistent state**: All users see same state

### ❌ Disadvantages
1. **Violates Single Responsibility**: Class controls both its logic and instantiation
2. **Hard to test**: Difficult to mock in unit tests
3. **Hidden dependencies**: Not clear from function signatures
4. **Global state**: Can lead to unexpected behavior
5. **Thread safety**: Requires careful implementation
6. **Tight coupling**: Code becomes dependent on singleton

## Python-Specific Alternatives

### 1. Module-Level Instance (Recommended)
```python
# config.py
class _Config:
    pass

config = _Config()  # Single instance

# Usage: from config import config
```

### 2. Dependency Injection
```python
# Better for testing!
class Service:
    def __init__(self, config):
        self.config = config  # Inject dependency
```

### 3. Context Manager
```python
with DatabaseConnection() as conn:
    # Use connection
    pass
```

## When NOT to Use Singleton

❌ **Don't use for:**
- Simple utility functions (use module-level functions)
- Configuration that changes per test (use dependency injection)
- When you need multiple instances for testing
- When the "single instance" is just a convenience, not a requirement

## Best Practices

1. **Prefer module-level instances** for Python code
2. **Use dependency injection** for better testability
3. **Document** that a class is a singleton
4. **Thread safety** if used in multithreaded environment
5. **Consider alternatives** before using singleton

## Summary

The Singleton pattern ensures only one instance exists, but should be used sparingly. In Python:
- Module-level instances are often sufficient
- Metaclass approach is most Pythonic for true singletons
- Consider dependency injection for better testability
- Be aware of the downsides (testing, coupling, hidden dependencies)

**Remember**: "If you wonder whether you need a Singleton, you probably don't." - Consider simpler alternatives first!